<a href="https://colab.research.google.com/github/KimutaiLawrence/Geospatial-Data-Science/blob/main/Accessing_Satellite_Imagery_Using%C2%A0Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Accessing data: We will use Sentinel 2 data. There are many options to access Sentinel 2 images and 
most of them will require you to access through website interaction whether directly via a downloading service utility 
or via the cloud. However, since we are using Jupyter notebook, we will access them right here using, sentinelsat a python 
library which makes searching, retrieving and downloading Sentinel satellite images easy. So let us start installing 
sentinelsat through pip. We also install other packages that we will use as we continue.

In [ ]:
!pip install sentinelsat
!pip install folium
!pip install descartes
!pip install rasterio
!pip install wget
!pip install geopandas
!pip install pandas

     |████████████████████████████████| 48 kB 3.2 MB/s 
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=de09231f74583b605bd0993c95abab63d7f8d9cf15b13aeb55b290004e45ba3c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 1.0 MB 7.8 MB/s 
     |████████████████████████████████| 16.7 MB 38.5 MB/s 
     |████████████████████████████████| 6.3 MB 36.3 MB/s 


In [ ]:
import folium
import os
import numpy as np

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt

from shapely.geometry import MultiPolygon, Polygon
import fiona
import wget

Before we are able to use sentinelsat, we need to register a username in Copernicus Open Access Hub and note down your username 
and password and paste them here inside the code.

In [ ]:
from sentinelsat import SentinelAPI

user = 'lawrence65' 
password = 'lawrence65' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

You are set to use sentinelsat and download Sentinel Satellite images.We then use boundary data from Lagos City, Nigeria. I uploaded the boundary for Lagos city,Nigeria here on google colab . We thus access the boundary data(Lagos shapefile) via the Lagos shapefile path and we will read it with Geopandas and visualize it with Folium python library.

In [ ]:

print("Installing geopandas...")

# We need to install geopandas and descartes using PIP because they are 
# not installed on Jupyter by default. 

!pip install geopandas
!pip install descartes 

import geopandas as gpd

# Next, print out the shapefile. 
# It won't look like much because we are looking at geographic coordinates. 

print("Loading Shapefile...")

# If using your files, replace below filename ("/content/Lagosnigeria.shp") with the 
# shapefile filename you uploaded. 

shapefile = gpd.read_file("/content/Lagosnigeria.shp")

# The "head" function prints out the first five rows in full, so you can see
# the columns in the data set too! 

shapefile.head()

Installing geopandas...
Loading Shapefile...


,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
0,163,NGA,Nigeria,25,Lagos,506,Agege,Local Authority,Local Authority,None,None,"POLYGON ((3.33228 6.64879, 3.33108 6.64763, 3...."
1,163,NGA,Nigeria,25,Lagos,507,Ajeromi/Ifelodun,Local Authority,Local Authority,None,None,"POLYGON ((3.32256 6.44672, 3.32138 6.44733, 3...."
2,163,NGA,Nigeria,25,Lagos,508,Alimosho,Local Authority,Local Authority,None,None,"POLYGON ((3.28883 6.58545, 3.28600 6.58480, 3...."
3,163,NGA,Nigeria,25,Lagos,509,Amuwo Odofin,Local Authority,Local Authority,None,None,"POLYGON ((3.18543 6.40375, 3.18554 6.40677, 3...."
4,163,NGA,Nigeria,25,Lagos,510,Apapa,Local Authority,Local Authority,None,None,"POLYGON ((3.32256 6.44672, 3.32486 6.44819, 3...."


One last step before we can search and download sentinel 2 images is to create a footprint from the Lagos Shapefile Geometry. 
Here we will use Shapely Python library since our data is in Shapefiles and have read it already as Geopandas GeodataFrame. 
(Note that if you have Geojson data, sentinelsatprovides a handy way to convert your data into a proper format in the query).

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in shapefile['geometry']:
    footprint = i

Now we can run a query on the api we have created above. There are different ways you can construct your query here depending on your use case. In this example, we will create a query for Sentinel 2 images Level 2A with cloud coverage between 0 and 10 
that fall or intersect with the footprint (Area of study:"in our case Lagos City"). For the time period, we are interested only in Sentinel Level 2A satellite images taken between ‘20220101’ and ‘20220120’ (For reference on valid search queries please refer to scihub).

In [ ]:
products = api.query(footprint,
                     date = ('20190101', '20190110'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10)
                    )

Sorting the Sentinel 2 data 

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted

,title,link,link_alternative,link_icon,summary,ondemand,ingestiondate,beginposition,endposition,orbitnumber,...,platformidentifier,orbitdirection,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,uuid,geometry
4f92eb8c-4239-4364-9ffc-a0b0bbabc535,S2B_MSIL2A_20190106T100409_N0211_R122_T31NEH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-01-06T10:04:09.024Z, Instrument: MS...",false,2019-01-06 20:53:21.770,2019-01-06 10:04:09.024,2019-01-06 10:04:09.024,9586,...,2017-013A,DESCENDING,Sentinel-2B,02.11,Level-2A,S2MSI2A,Sentinel-2,1.02 GB,4f92eb8c-4239-4364-9ffc-a0b0bbabc535,"POLYGON ((2.99982 7.23783, 3.99439 7.23674, 3...."
ab40b37d-d1ae-4592-bb01-3ed0adf2527a,S2A_MSIL2A_20190101T100411_N0211_R122_T31NEH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-01-01T10:04:11.024Z, Instrument: MS...",false,2019-01-01 15:54:36.102,2019-01-01 10:04:11.024,2019-01-01 10:04:11.024,18423,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1.02 GB,ab40b37d-d1ae-4592-bb01-3ed0adf2527a,"POLYGON ((2.99982 7.23783, 3.99439 7.23674, 3...."


From the results that are acquired above, you can now download your Sentinel 2 data (specific) from the list of outputs.

In [ ]:
api.download("ab40b37d-d1ae-4592-bb01-3ed0adf2527a")

MD5 checksumming:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

{'Creation Date': datetime.datetime(2019, 1, 1, 16, 3, 17, 192000),
 'Ingestion Date': datetime.datetime(2019, 1, 1, 15, 54, 36, 102000),
 'Online': True,
 'date': datetime.datetime(2019, 1, 1, 10, 4, 11, 24000),
 'downloaded_bytes': 1099405101,
 'footprint': 'POLYGON((2.999818830996764 7.23782745247485,3.994385617464529 7.236741842485151,3.992363882297123 6.243634699399232,2.999819199414278 6.244570099446785,2.999818830996764 7.23782745247485))',
 'id': 'ab40b37d-d1ae-4592-bb01-3ed0adf2527a',
 'md5': 'b53789caf34ab20839aea9503364c0eb',
 'path': 'S2A_MSIL2A_20190101T100411_N0211_R122_T31NEH_20190101T122648.zip',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('ab40b37d-d1ae-4592-bb01-3ed0adf2527a')/Products('Quicklook')/$value",
 'size': 1099405101,
 'title': 'S2A_MSIL2A_20190101T100411_N0211_R122_T31NEH_20190101T122648',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('ab40b37d-d1ae-4592-bb01-3ed0adf2527a')/$value"}